In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
icon_info = []  # ICON 선수들의 정보를 담을 리스트
moment_info = []    # MOMENT 선수들의 정보를 담을 리스트

In [3]:
# 웹 크롤링 시작 (셀레니움)
driver = webdriver.Chrome()
driver.get('https://fconline.nexon.com/datacenter')

# ICON 선수들의 정보(이름, 왼발 + 오른발, 평점)를 크롤링
element = driver.find_element(By.XPATH, '//*[@id="form1"]/div[2]/div[1]/div[2]/div[3]')
driver.execute_script("arguments[0].scrollIntoView(true);", element)
element.click()

driver.find_element(By.XPATH, '//*[@id="form1"]/div[1]/div[3]/button[1]').click()
time.sleep(3)

for i in range(1, 135):
    name = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[1]/div/div[4]/div[2]').text  # 이름
    feet = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[1]/div/div[5]/div').text
    feet = int(feet[1]) + int(feet[-1]) # 왼발 + 오른발
    score = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[9]/span').text
    score = score.split()[0]    # 평점
    icon_info.append((name, feet, score))

driver.find_element(By.XPATH, '//*[@id="form1"]/div[2]/div[1]/div[2]/div[3]').click()
time.sleep(3)

# MOMENT 선수들의 정보(이름, 왼발 + 오른발, 평점)를 크롤링
element = driver.find_element(By.XPATH, '//*[@id="form1"]/div[2]/div[1]/div[2]/div[1]')
driver.execute_script("arguments[0].scrollIntoView(true);", element)
element.click()

driver.find_element(By.XPATH, '//*[@id="form1"]/div[1]/div[3]/button[1]').click()
time.sleep(3)

for i in range(1, 113):
    name = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[1]/div/div[4]/div[2]').text  # 이름
    feet = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[1]/div/div[5]/div').text
    feet = int(feet[1]) + int(feet[-1]) # 왼발 + 오른발
    score = driver.find_element(By.XPATH, '//*[@id="divPlayerList"]/div['+str(i)+']/div[9]/span').text
    score = score.split()[0]    # 평점
    moment_info.append((name, feet, score))

driver.quit()   # 크롤링 종료

In [4]:
# MOMENT 명단에 없는 ICON 선수들은 ICON 정보에서 제외
moment_name = {player[0] for player in moment_info}
filtered_icon_info = [player for player in icon_info if player[0] in moment_name]

In [7]:
# ICON, MOMENT 선수 정보를 각각의 데이터프레임으로 생성
df1 = pd.DataFrame(filtered_icon_info, columns=['이름', '발(아이콘)', '평점(아이콘)'])
df2 = pd.DataFrame(moment_info, columns=['이름', '발(모먼트)', '평점(모먼트)'])

# df1과 df2를 이름 컬럼을 기준으로 조인 병합
df = pd.merge(df1, df2, on='이름')
df.head(10)

,이름,발(아이콘),평점(아이콘),발(모먼트),평점(모먼트)
0,펠레,10,8.6,10,6.8
1,지네딘 지단,10,9.2,10,8.5
2,호나우두,10,8.2,10,6.4
3,요한 크루이프,10,8.5,10,8.2
4,게르트 뮐러,10,8.6,10,8.7
5,로타어 마테우스,10,9.2,10,9.4
6,마르코 반바스텐,9,8.6,10,8.1
7,페렌츠 푸스카스,8,8.4,9,8.4
8,파올로 말디니,9,8.8,10,8.8
9,로베르토 바조,10,8.2,10,9.2


In [6]:
# 병합된 데이터프레임을 csv 파일로 저장
df.to_csv('C:\\Users\\이진호\\Desktop\\Programming\\프로젝트\\MyProject1\\players_info.csv', encoding='utf-8-sig', index = 0)